# 주식 및 ETF 데이터 수집

In [1]:
# pip install beautifulsoup4 pandas openpyxl selenium webdriver-manager pykrx sqlalchemy pymysql


In [1]:
# pip install pymysql


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'e:\아경\AI\Python38\python.exe -m pip install --upgrade pip' command.


In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
from pykrx import stock
import datetime
import os
from sqlalchemy import create_engine
import pymysql
import numpy as np
import configparser


In [2]:


# 설정 파일 읽기
config = configparser.ConfigParser()
config.read('config.ini')

username = config['mysql']['username']
password = config['mysql']['password']
host = config['mysql']['host']
database = config['mysql']['database']

conn = pymysql.connect(host=host, user=username, password=password, db=database, charset='utf8')
cur = conn.cursor()


In [14]:
# 테이블 생성 함수
def create_tables():
    cur.execute('''
    CREATE TABLE IF NOT EXISTS stock_data (
        ticker VARCHAR(10),
        name VARCHAR(100),
        date DATE,
        open FLOAT,
        high FLOAT,
        low FLOAT,
        close FLOAT,
        volume BIGINT,
        value BIGINT,
        market_cap BIGINT,
        shares_outstanding BIGINT,
        PER FLOAT,
        PBR FLOAT,
        dividend FLOAT,
        BPS FLOAT,
        EPS FLOAT,
        DPS FLOAT,
        normalized_value FLOAT,
        PRIMARY KEY (ticker, date)
    )
    ''')
    cur.execute('''
    CREATE TABLE IF NOT EXISTS ticker_list (
        ticker VARCHAR(10) PRIMARY KEY,
        market VARCHAR(10),
        name VARCHAR(100),
        last_updated DATE,
        is_delisted BOOLEAN DEFAULT FALSE
    )
    ''')
    cur.execute('''
    CREATE TABLE IF NOT EXISTS ticker_status (
        ticker VARCHAR(10) PRIMARY KEY,
        status VARCHAR(20)
    )
    ''')
create_tables()

In [15]:

# 모든 티커 리스트를 저장할 집합
all_tickers = set()

# 시작 날짜와 현재 날짜 설정
start_date = '19950102' # 이 시기부터 ticker list를 제공
end_date = datetime.datetime.now().strftime('%Y%m%d')

# 모든 날짜에 대해 티커 리스트 가져오기
current_date = datetime.datetime.strptime(start_date, '%Y%m%d')
while current_date.strftime('%Y%m%d') <= end_date:
    date_str = current_date.strftime('%Y%m%d')
    try:
        tickers_kosdaq = stock.get_market_ticker_list(date_str, market='KOSDAQ')
        tickers_kospi = stock.get_market_ticker_list(date_str, market='KOSPI')
        all_tickers.update(tickers_kosdaq)
        all_tickers.update(tickers_kospi)
        print(len(all_tickers),f'{date_str}')
        time.sleep(0.8)
    except Exception as e:
        print(f"Error on date {date_str}: {e}")
    current_date += datetime.timedelta(days=180)  # 180일 단위로 진행
    
# 티커 목록을 MySQL에 저장
for ticker in all_tickers:
    market = 'KOSDAQ' if ticker in tickers_kosdaq else 'KOSPI'
    name = stock.get_market_ticker_name(ticker)
    cur.execute('''
    INSERT INTO ticker_list (ticker, market, name) VALUES (%s, %s, %s)
    ON DUPLICATE KEY UPDATE market=VALUES(market), name=VALUES(name)
    ''', (ticker, market, name))
    # cur.execute('''
    # INSERT INTO ticker_status (ticker, status) VALUES (%s, 'pending')
    # ON DUPLICATE KEY UPDATE status=VALUES(status)
    # ''', (ticker,))  #1회 실행
    conn.commit()

cur.close()
conn.close()
print("모든 티커 목록이 성공적으로 저장되었습니다.")

0 19950102
956 19950701
1132 19951228
1190 19960625
1758 19961222
1827 19970620
1906 19971217
1930 19980615
1941 19981212
2003 19990610
2117 19991207
2298 20000604
2440 20001201
2525 20010530
2638 20011126
2773 20020525
2848 20021121
2895 20030520
2944 20031116
2983 20040514
3016 20041110
3061 20050509
3121 20051105
3164 20060504
3214 20061031
3257 20070429
3322 20071026
3361 20080423
3404 20081020
3428 20090418
3467 20091015
3519 20100413
3571 20101010
3625 20110408
3664 20111005
3702 20120402
3720 20120929
3740 20130328
3758 20130924
3785 20140323
3807 20140919
3865 20150318
3932 20150914
4002 20160312
4034 20160908
4087 20170307
4137 20170903
4189 20180302
4227 20180829
4292 20190225
4338 20190824
4406 20200220
4450 20200818
4517 20210214
4573 20210813
4639 20220209
4693 20220808
4762 20230204
4832 20230803
4908 20240130
모든 티커 목록이 성공적으로 저장되었습니다.


## Step 2: 데이터 갱신 및 새로운 티커 추가 (중단 후 재시작 가능)
* 한번 실행한 경우 몇 거래일 뒤에 다시 실행해야 오류가 발생하지 않음


In [8]:
import datetime
import pymysql
import configparser
from pykrx import stock
import numpy as np

# 설정 파일 읽기
config = configparser.ConfigParser()
config.read('config.ini')

username = config['mysql']['username']
password = config['mysql']['password']
host = config['mysql']['host']
database = config['mysql']['database']

conn = pymysql.connect(host=host, user=username, password=password, db=database, charset='utf8')
cur = conn.cursor()

# 조건 설정
per_threshold = 10
pbr_threshold = 1
div_threshold = 3.5  # 기준금리 참조

# 현재 날짜 기준으로 최근 월 종목 리스트 수집
current_date = datetime.datetime.now()
last_month_str = current_date.strftime("%Y%m") + '01'

# 데이터베이스에서 모든 티커 가져오기
cur.execute("SELECT ticker FROM ticker_list")
all_tickers = [row[0] for row in cur.fetchall()]

# 각 티커에 대한 처리
for ticker in all_tickers:
    # 이미 처리된 티커는 스킵
    cur.execute(f"SELECT status FROM ticker_status WHERE ticker = '{ticker}'")
    status = cur.fetchone()
    if status and status[0] == 'completed':
        continue
    
     # 티커의 끝자리가 0이 아닌 경우 스킵
    if ticker[-1] != '0':
        print(f'{ticker} skipped because it does not end with 0')
        continue    
    
    # 데이터베이스에서 마지막 날짜 확인
    cur.execute(f"SELECT MAX(date) FROM stock_data WHERE ticker = '{ticker}'")
    last_date = cur.fetchone()[0]
    # 마지막 날짜가 현재 날짜와 7일 이하로 차이나는 경우 continue
    if last_date:
        last_recorded_date = pd.to_datetime(last_date)
        if (current_date - last_recorded_date).days <= 7:
            print(f'{ticker} already done')
            continue
    # 새 데이터를 가져옴
    if last_date:
        start_date = (last_date + datetime.timedelta(days=1)).strftime("%Y%m%d")
    else:
        start_date = "19800102"

    end_date = current_date.strftime("%Y%m%d")
    time.sleep(1)
    df2 = stock.get_market_fundamental(start_date, end_date, ticker)


    if df2.empty or 'PER' not in df2.columns:
        cur.execute(f"UPDATE ticker_status SET status = 'completed' WHERE ticker = '{ticker}'")
        conn.commit()
        print(f'{ticker} no.2 pass condition')
        continue

    # 조건 확인
    if ('PER' in df2.columns and 'PBR' in df2.columns and 'DIV' in df2.columns):
        condition = (df2['PER'] > 0) & (df2['PER'] <= per_threshold) & \
                    (df2['PBR'] > 0) & (df2['PBR'] <= pbr_threshold) & \
                    (df2['DIV'] >= div_threshold)
        if not condition.any():
            cur.execute(f"UPDATE ticker_status SET status = 'completed' WHERE ticker = '{ticker}'")
            conn.commit()
            print(f'{ticker} no.3 pass condition')
            time.sleep(0.8)  
            continue  # 조건을 만족하지 않으면 다음 티커로 넘어감
        
    else:
        cur.execute(f"UPDATE ticker_status SET status = 'completed' WHERE ticker = '{ticker}'")
        conn.commit()
        print(f'{ticker} no.4 pass condition')
        continue  # 필요한 컬럼이 없으면 넘어감
    time.sleep(0.8)    
    df1 = stock.get_market_ohlcv(start_date, end_date, ticker)
    

    if df1.empty:
        cur.execute(f"UPDATE ticker_status SET status = 'completed' WHERE ticker = '{ticker}'")
        conn.commit()
        print(f'{ticker} no.5 pass condition')
        continue

    # '고가' 컬럼이 있는지 확인
    if '고가' not in df1.columns:
        cur.execute(f"UPDATE ticker_status SET status = 'completed' WHERE ticker = '{ticker}'")
        conn.commit()
        print(f'{ticker} no.6 pass condition')
        continue
    time.sleep(1.2)   
    df3 = stock.get_market_cap(start_date, end_date, ticker)
    

    combined_df = pd.concat([df1, df2, df3], axis=1, join='outer')

    # 종목명 가져오기
    time.sleep(0.5)
    name = stock.get_market_ticker_name(ticker)
    time.sleep(0.8)

    # 노멀라이즈드 값 계산
    combined_df['normalized_value'] = (combined_df['종가'] - combined_df['종가'].rolling(window=252*5, min_periods=252).min()) / \
                                      (combined_df['종가'].rolling(window=252*5, min_periods=252).max() - combined_df['종가'].rolling(window=252*5, min_periods=252).min()) * 100

    combined_df.reset_index(inplace=True)
    combined_df['ticker'] = ticker
    combined_df['name'] = name
    combined_df.rename(columns={
        '날짜': 'date',
        '시가': 'open',
        '고가': 'high',
        '저가': 'low',
        '종가': 'close',
        '거래량': 'volume',
        '거래대금': 'value',
        '시가총액': 'market_cap',
        '상장주식수': 'shares_outstanding',
        'DIV': 'dividend',
        'BPS': 'BPS',
        'EPS': 'EPS',
        'DPS': 'DPS'
    }, inplace=True)

    # 중복된 'volume' 열 제거
    combined_df = combined_df.loc[:, ~combined_df.columns.duplicated()]

    # NaN 값을 None으로 변환 (개별적으로 처리)
    combined_df = combined_df.replace({np.nan: None})

    # 데이터베이스에 저장
    for _, row in combined_df.iterrows():
        sql = '''
        INSERT INTO stock_data (ticker, name, date, open, high, low, close, volume, value, market_cap, shares_outstanding, PER, PBR, dividend, BPS, EPS, DPS, normalized_value)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        ON DUPLICATE KEY UPDATE
            open=VALUES(open),
            high=VALUES(high),
            low=VALUES(low),
            close=VALUES(close),
            volume=VALUES(volume),
            value=VALUES(value),
            market_cap=VALUES(market_cap),
            shares_outstanding=VALUES(shares_outstanding),
            PER=VALUES(PER),
            PBR=VALUES(PBR),
            dividend=VALUES(dividend),
            BPS=VALUES(BPS),
            EPS=VALUES(EPS),
            DPS=VALUES(DPS),
            normalized_value=VALUES(normalized_value)
        '''
        cur.execute(sql, (
            row['ticker'],
            row['name'],
            row['date'].strftime('%Y-%m-%d') if row['date'] else None,
            row['open'],
            row['high'],
            row['low'],
            row['close'],
            int(row['volume']) if row['volume'] is not None else None,
            int(row['value']) if row['value'] is not None else None,
            int(row['market_cap']) if row['market_cap'] is not None else None,
            int(row['shares_outstanding']) if row['shares_outstanding'] is not None else None,
            row['PER'],
            row['PBR'],
            row['dividend'],
            row['BPS'],
            row['EPS'],
            row['DPS'],
            row['normalized_value']
        ))

    conn.commit()
     # 최근 60일 동안 데이터가 있는지 확인
    last_recorded_date = pd.to_datetime(combined_df['date']).max()
    if (current_date - last_recorded_date).days > 60:
        print(current_date - last_recorded_date,current_date,last_recorded_date)
        cur.execute(f"UPDATE ticker_status SET status = 'completed' WHERE ticker = '{ticker}'")
        conn.commit()
    print(f"{ticker} 데이터베이스에 저장 완료")
    time.sleep(0.5)


cur.close()
conn.close()
print("모든 데이터가 성공적으로 저장되었습니다.")

000020 already done
000070 already done
000140 already done
000150 already done
000210 already done
000220 already done
000230 already done
000240 already done
000250 already done
000270 already done
000320 already done
000370 already done
000390 already done
000400 already done
000430 already done
000440 already done
000480 already done
000490 already done
000500 already done
000520 already done
000590 already done
000640 already done
000700 already done
000721 skipped because it does not end with 0
000725 skipped because it does not end with 0
000726 skipped because it does not end with 0
000791 skipped because it does not end with 0
000794 skipped because it does not end with 0
000801 skipped because it does not end with 0
000805 skipped because it does not end with 0
000810 already done
000815 skipped because it does not end with 0
000831 skipped because it does not end with 0
000835 skipped because it does not end with 0
000850 already done
000860 already done
000880 already done


ConnectionError: HTTPConnectionPool(host='data.krx.co.kr', port=80): Max retries exceeded with url: /comm/bldAttendant/getJsonData.cmd (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000264D5402400>: Failed to establish a new connection: [WinError 10060] 연결된 구성원으로부터 응답이 없어 연결하지 못했거나, 호스트로부터 응답이 없어 연결이 끊어졌습니다'))

## ETF 이름 & 티커 리스트 생성  

In [6]:
# 크롬 드라이버 설정
options = webdriver.ChromeOptions()
options.add_argument('headless')  # 브라우저 창을 띄우지 않음
options.add_argument('disable-gpu')  # GPU 가속 비활성화
options.add_argument('lang=ko_KR')  # 한국어 페이지

# 크롬 드라이버 초기화
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

In [6]:
import pandas as pd
import pymysql
import configparser
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import time

# 설정 파일 읽기
config = configparser.ConfigParser()
config.read('config.ini')

username = config['mysql']['username']
password = config['mysql']['password']
host = config['mysql']['host']
database = config['mysql']['database']

# MySQL 연결
conn = pymysql.connect(host=host, user=username, password=password, db=database, charset='utf8')
cur = conn.cursor()

# 크롬 드라이버 설정
options = webdriver.ChromeOptions()
options.add_argument('headless')  # 브라우저 창을 띄우지 않음
options.add_argument('disable-gpu')  # GPU 가속 비활성화
options.add_argument('lang=ko_KR')  # 한국어 페이지

# 크롬 드라이버 초기화
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
# URL에서 HTML 컨텐츠를 가져옴
url = 'https://finance.naver.com/sise/etf.naver'
driver.get(url)

# 종목명과 티커를 추출하여 리스트로 저장
etf_data = []
rows = driver.find_elements(By.CSS_SELECTOR, '#etfItemTable > tr')
for row in rows:
    name_col = row.find_elements(By.CSS_SELECTOR, 'td.ctg a')
    if name_col:
        name = name_col[0].text.strip()
        ticker = name_col[0].get_attribute('href').split('=')[-1]  # 종목명 링크에서 티커 추출
        etf_data.append([name, ticker])

# 드라이버 종료
driver.quit()

# DataFrame 생성
df = pd.DataFrame(etf_data, columns=['종목명', '티커'])

# DataFrame을 Excel 파일로 저장
excel_filename = 'etf_list.xlsx'
df.to_excel(excel_filename, index=False)
print(f"Excel 파일로 저장되었습니다: {excel_filename}")

# DataFrame을 MySQL에 저장
create_table_query = '''
CREATE TABLE IF NOT EXISTS etf_list (
    name VARCHAR(100),
    ticker VARCHAR(20),
    PRIMARY KEY (ticker)
)
'''
cur.execute(create_table_query)
conn.commit()

# MySQL에 데이터 삽입
for index, row in df.iterrows():
    insert_query = '''
    INSERT INTO etf_list (name, ticker) VALUES (%s, %s)
    ON DUPLICATE KEY UPDATE name=VALUES(name)
    '''
    cur.execute(insert_query, (row['종목명'], row['티커']))

conn.commit()
cur.close()
conn.close()
print("MySQL 데이터베이스에 저장되었습니다.")

Excel 파일로 저장되었습니다: etf_list.xlsx
MySQL 데이터베이스에 저장되었습니다.
